In [ ]:
!pip install transformers pandas scikit-learn

In [ ]:
#3: Stress Detection Pipeline
from transformers import pipeline

# Using a pre-trained sentiment analysis model, which serves as a proxy for stress detection
# 'finiteautomata/bertweet-base-sentiment-analysis' is a fast, good option on a free Colab GPU.
stress_detector = pipeline(
    'sentiment-analysis',# Task type: sentiment analysis
    model='finiteautomata/bertweet-base-sentiment-analysis',# One of Google's BERT model
    device=0 # Use GPU if available, use -1 for CPU
)

def detect_stress_score(text):
    """Classifies text and returns a 'stress score' based on negative sentiment."""
    result = stress_detector(text)[0]
    label = result['label']
    score = result['score']

    # Map negative sentiment to a stress score (higher is more stressed)
    if 'NEG' in label:
        # Scale the confidence of negative sentiment
        stress_score = score
    elif 'NEU' in label:
        # Moderate stress/neutral tone
        stress_score = 0.5 * score
    else: # POS
        stress_score = 1.0 - score # Invert positive confidence for a low score

    return stress_score, label, round(score, 4)

# Test the model
test_text_1 = "I am so overwhelmed with this final project."
test_text_2 = "Excited to start the new module!"

score1, label1, conf1 = detect_stress_score(test_text_1)
score2, label2, conf2 = detect_stress_score(test_text_2)

print(f"Post: '{test_text_1}' -> Label: {label1}, Confidence: {conf1}, Stress Score: {round(score1, 4)}")
print(f"Post: '{test_text_2}' -> Label: {label2}, Confidence: {conf2}, Stress Score: {round(score2, 4)}")

# Define the stress threshold
STRESS_THRESHOLD = 0.70 # Can be adjusted

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/949 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/540M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/540M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/338 [00:00<?, ?B/s]

...

bpe.codes: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/22.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0
Device set to use cuda:0


Post: 'I am so overwhelmed with this final project.' -> Label: NEG, Confidence: 0.5167, Stress Score: 0.5167
Post: 'Excited to start the new module!' -> Label: POS, Confidence: 0.993, Stress Score: 0.007


In [ ]:
# Colab Code Snippet 4: Recommendation System
SUPPORT_RESOURCES = {
    'HIGH': {
        'message': "🚨 Immediate Concern: This suggests high distress. Please reach out to your university's *Counseling Services* immediately.",
        'links': ["Social Emotional Learning and Psychological Counselling: 8891091149", "Contact Ms.Merlyn Sarah at merlynandrews@karunya.edu"]
    },
    'MEDIUM': {
        'message': "❗ Timely Intervention: We recommend reviewing time management techniques and reaching out to a *Peer Mentor* or *Academic Advisor* for help with course material.",
        'links': ["Karunya Wellness Centre", "Call:9750768466 or mail: joycelinmary@karunya.edu"]
    },
    'LOW': {
        'message': "✅ Proactive Tip: Keep up the great work! If you feel stuck, remember to take short breaks.",
        'links': ["https://www.nhs.uk/mental-health/self-help/guides-tools-and-activities/five-steps-to-mental-wellbeing/"]
    }
}

def provide_recommendation(stress_score):
    """Provides a resource based on the calculated stress score."""
    if stress_score >= 0.85:
        level = 'HIGH'
    elif stress_score >= STRESS_THRESHOLD: # Threshold set in previous step (e.g., 0.70)
        level = 'MEDIUM'
    else:
        level = 'LOW'

    resource = SUPPORT_RESOURCES[level]
    print(f"\n--- Support Recommendation ({level}) ---")
    print(resource['message'])
    print("Suggested Links:")
    for link in resource['links']:
        print(f"- {link}")
    print("---------------------------------------")

In [ ]:
# Colab Code Snippet 5: Main Function
def monitor_and_intervene(forum_post):
    """
    Runs the E2E stress detection and intervention process.
    """
    print(f"==================================================")
    print(f"New Forum Post Received: \"{forum_post}\"")
    print(f"==================================================")

    # 1. Stress Detection
    stress_score, sentiment_label, confidence = detect_stress_score(forum_post)
    stress_score = round(stress_score, 4)

    print(f"Model Analysis: Primary Sentiment='{sentiment_label}' with Confidence={confidence}")
    print(f"Calculated Stress Score: {stress_score}")

    # 2. Intervention Logic
    if stress_score >= STRESS_THRESHOLD:
        print(f"🚨 STRESS DETECTED: (Score > {STRESS_THRESHOLD}) - INITIATING INTERVENTION.")
        provide_recommendation(stress_score)
    else:
        print("✅ Stress score is low. No immediate intervention needed.")

    print(f"==================================================\n")

# --- DEMONSTRATION ---
print("--- DEMO 1: HIGH STRESS POST ---")
monitor_and_intervene("I just can't handle this workload anymore. I feel completely hopeless about passing this semester.")

print("--- DEMO 2: MEDIUM STRESS POST ---")
monitor_and_intervene("Is anyone else finding the project brief impossible to understand? I'm getting really frustrated.")

print("--- DEMO 3: LOW STRESS POST ---")
monitor_and_intervene("I think the answer to question 5 is 'C'. Does anyone agree or disagree?")

--- DEMO 1: HIGH STRESS POST ---
New Forum Post Received: "I just can't handle this workload anymore. I feel completely hopeless about passing this semester."
Model Analysis: Primary Sentiment='NEG' with Confidence=0.9823
Calculated Stress Score: 0.9823
🚨 STRESS DETECTED: (Score > 0.7) - INITIATING INTERVENTION.

--- Support Recommendation (HIGH) ---
🚨 Immediate Concern: This suggests high distress. Please reach out to your university's *Counseling Services* immediately.
Suggested Links:
- Social Emotional Learning and Psychological Counselling: 8891091149
- Contact Ms.Merlyn Sarah at merlynandrews@karunya.edu
---------------------------------------

--- DEMO 2: MEDIUM STRESS POST ---
New Forum Post Received: "Is anyone else finding the project brief impossible to understand? I'm getting really frustrated."
Model Analysis: Primary Sentiment='NEG' with Confidence=0.9796
Calculated Stress Score: 0.9796
🚨 STRESS DETECTED: (Score > 0.7) - INITIATING INTERVENTION.

--- Support Recommendatio

In [ ]:
# Colab Code Snippet 6: Upgraded Stress Detection using LLM (FIXED)

# !pip install transformers torch accelerate pandas scikit-learn

import torch
from transformers import pipeline
from transformers.utils import is_flash_attn_2_available

# Define the recommendation function if it wasn't defined earlier (Copy of Snippet 4 logic)
SUPPORT_RESOURCES = {
    'HIGH': {
        'message': "🚨 Immediate Concern: This suggests high distress. Please reach out to your university's *Counseling Services* immediately.",
        'links': ["Social Emotional Learning and Psychological Counselling: 8891091149", "Contact Ms.Merlyn Sarah at merlynandrews@karunya.edu"]
    },
    'MEDIUM': {
        'message': "❗ Timely Intervention: We recommend reviewing time management techniques and reaching out to a *Peer Mentor* or *Academic Advisor* for help with course material.",
        'links': ["Karunya Wellness Centre", "Call:9750768466 or mail: joycelinmary@karunya.edu"]
    },
    'LOW': {
        'message': "✅ Proactive Tip: Keep up the great work! If you feel stuck, remember to take short breaks.",
        'links': ["https://www.nhs.uk/mental-health/self-help/guides-tools-and-activities/five-steps-to-mental-wellbeing/"]
    }
}

def provide_recommendation(stress_score):
    STRESS_THRESHOLD = 0.65
    if stress_score >= 0.85:
        level = 'HIGH'
    elif stress_score >= STRESS_THRESHOLD:
        level = 'MEDIUM'
    else:
        level = 'LOW'

    resource = SUPPORT_RESOURCES[level]
    print(f"\n--- Support Recommendation ({level}) ---")
    print(resource['message'])
    print("Suggested Links:")
    for link in resource['links']:
        print(f"- {link}")
    print("---------------------------------------")

print("Loading specialized mental health LLM...")

try:
    # Use a pre-trained LLM for zero-shot text classification
    stress_llm = pipeline(
        "text-generation",
        model="HuggingFaceH4/zephyr-7b-beta", # Model zephyr-7b LLM
        torch_dtype=torch.bfloat16,
        device_map="auto" # Use GPU automatically
    )

    # New prompt: Asking the LLM to act as an expert and classify the stress level
    def detect_stress_llm(text):
        system_prompt = (
            "You are an AI stress monitoring system in an academic forum. "
            "Analyze the following student post for signs of stress, anxiety, or being overwhelmed. "
            "Respond ONLY with one of the following words: HIGH_STRESS, MEDIUM_STRESS, or LOW_STRESS. DO NOT include any other text."
        )

        # Use the standard chat template for instruction models
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": text}
        ]

        # Tokenize and generate response
        response = stress_llm(messages, max_new_tokens=5, do_sample=False, truncation=True)

        # The output is a list of dicts; extract the generated text
        # Clean the output to get the pure classification word
        classification_text = response[0]['generated_text'][-1]['content'].strip().upper()

        if "HIGH_STRESS" in classification_text:
            return 0.90, "HIGH"
        elif "MEDIUM_STRESS" in classification_text:
            return 0.75, "MEDIUM"
        else:
            return 0.40, "LOW"

    # --- MAIN MONITOR FUNCTION (using the new LLM model) ---

    def monitor_and_intervene_v2(forum_post):
        """
        Runs the E2E stress detection and intervention process using the LLM.
        """
        print(f"==================================================")
        print(f"New Forum Post Received: \"{forum_post}\"")
        print(f"==================================================")

        # 1. Stress Detection using LLM
        stress_score, level_label = detect_stress_llm(forum_post)

        print(f"LLM Classification: {level_label}")
        print(f"Calculated Stress Score: {round(stress_score, 4)}")

        # 2. Intervention Logic
        provide_recommendation(stress_score)

        print(f"==================================================\n")


    # --- DEMONSTRATION OF UPGRADED MODEL ---
    STRESS_THRESHOLD = 0.65

    print("\n--- DEMO 1: HIGH STRESS POST (LLM) ---")
    monitor_and_intervene_v2("I'm failing the course and I'm honestly thinking about dropping out. This pressure is too much.")

    print("\n--- DEMO 2: MEDIUM STRESS POST (LLM) ---")
    monitor_and_intervene_v2("The recent assignment load is crushing my ability to sleep. I wish I had more time.")

    print("\n--- DEMO 3: LOW STRESS POST (LLM) ---")
    monitor_and_intervene_v2("Can someone confirm the due date for the history paper? I think it's Friday.")


except Exception as e:
    print(f"Could not load the LLM model due to resource constraints or other error: {e}")
    print("--------------------------------------------------------------------------------")
    print("🚨 **Action:** Falling back to the original Sentiment Model (Snippet 5).")
    print("--------------------------------------------------------------------------------")

Loading specialized mental health LLM...


config.json:   0%|          | 0.00/638 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 8 files:   0%|          | 0/8 [00:00<?, ?it/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

Device set to use cuda:0
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



--- DEMO 1: HIGH STRESS POST (LLM) ---
New Forum Post Received: "I'm failing the course and I'm honestly thinking about dropping out. This pressure is too much."
LLM Classification: HIGH
Calculated Stress Score: 0.9

--- Support Recommendation (HIGH) ---
🚨 Immediate Concern: This suggests high distress. Please reach out to your university's *Counseling Services* immediately.
Suggested Links:
- Social Emotional Learning and Psychological Counselling: 8891091149
- Contact Ms.Merlyn Sarah at merlynandrews@karunya.edu
---------------------------------------


--- DEMO 2: MEDIUM STRESS POST (LLM) ---
New Forum Post Received: "The recent assignment load is crushing my ability to sleep. I wish I had more time."
LLM Classification: HIGH
Calculated Stress Score: 0.9

--- Support Recommendation (HIGH) ---
🚨 Immediate Concern: This suggests high distress. Please reach out to your university's *Counseling Services* immediately.
Suggested Links:
- Social Emotional Learning and Psychological Counse

In [ ]:
!pip install gradio transformers torch
import gradio as gr
# ... [Place the definitions for monitor_and_intervene_v1 and monitor_and_intervene_v2 here] ...
# NOTE: monitor_and_intervene_v1 should be the BERT/Sentiment model (Snippet 5)
# NOTE: monitor_and_intervene_v2 should be the LLM model (Snippet 6 - Fixed)
# NOTE: Both functions need to return a single STRING containing the formatted output
#       (Stress Score + Recommendation) for the Gradio interface.

In [ ]:
import sys
from io import StringIO

def wrapper_v1(forum_post):
    # Capture print output
    old_stdout = sys.stdout
    redirected_output = sys.stdout = StringIO()
    monitor_and_intervene_v1(forum_post) # Your original function call
    sys.stdout = old_stdout
    # Return the captured string
    return redirected_output.getvalue()

def wrapper_v2(forum_post):
    # Capture print output
    old_stdout = sys.stdout
    redirected_output = sys.stdout = StringIO()
    monitor_and_intervene_v2(forum_post) # Your original function call
    sys.stdout = old_stdout
    # Return the captured string
    return redirected_output.getvalue()

In [ ]:
# ----------------------------------------------------------------------
# PART 1: SETUP AND MODEL DEFINITIONS
# ----------------------------------------------------------------------

import gradio as gr
import sys
from io import StringIO
import torch
from transformers import pipeline

# --- Model V1 Initialization (Basic Sentiment Model) ---
print("Initializing Basic Sentiment Model (V1)...")
# Note: If you get a 'device=0' error, remove it and let it use CPU/default device.
try:
    stress_detector = pipeline(
        'sentiment-analysis',
        model='finiteautomata/bertweet-base-sentiment-analysis',
        device=0 # Attempt to use GPU
    )
except Exception:
    stress_detector = pipeline(
        'sentiment-analysis',
        model='finiteautomata/bertweet-base-sentiment-analysis',
    )


# Define the support resources (Constant)
SUPPORT_RESOURCES = {
    'HIGH': {
        'message': "🚨 Immediate Concern: This suggests high distress. Please reach out to your university's *Counseling Services* immediately.",
        'links': ["Social Emotional Learning and Psychological Counselling: 8891091149", "Contact Ms.Merlyn Sarah at merlynandrews@karunya.edu"]
    },
    'MEDIUM': {
        'message': "❗ Timely Intervention: We recommend reviewing time management techniques and reaching out to a *Peer Mentor* or *Academic Advisor* for help with course material.",
        'links': ["Karunya Wellness Centre", "Call:9750768466 or mail: joycelinmary@karunya.edu"]
    },
    'LOW': {
        'message': "✅ Proactive Tip: Keep up the great work! If you feel stuck, remember to take short breaks.",
        'links': ["https://www.nhs.uk/mental-health/self-help/guides-tools-and-activities/five-steps-to-mental-wellbeing/"]
    }
}

def provide_recommendation(stress_score, STRESS_THRESHOLD):
    """Provides a resource based on the calculated stress score."""
    # Use Markdown formatting in the output string
    if stress_score >= 0.85:
        level = 'HIGH'
    elif stress_score >= STRESS_THRESHOLD:
        level = 'MEDIUM'
    else:
        level = 'LOW'

    resource = SUPPORT_RESOURCES[level]
    output = f"\n<p style='font-size: 1.1em; color: {'red' if level == 'HIGH' else 'orange' if level == 'MEDIUM' else 'green'}; font-weight: bold;'>{resource['message']}</p>\n"
    output += "Suggested Resources:\n"
    for link in resource['links']:
        output += f"- {link}\n"
    return output


# --- Model V1 Functions (Basic Sentiment) ---

def detect_stress_score(text):
    """Classifies text and returns a 'stress score' based on negative sentiment."""
    result = stress_detector(text)[0]
    label = result['label']
    score = result['score']

    if 'NEG' in label:
        stress_score = score
    elif 'NEU' in label:
        stress_score = 0.5 * score
    else: # POS
        stress_score = 1.0 - score
    return stress_score, label, round(score, 4)


def monitor_and_intervene_v1(forum_post, STRESS_THRESHOLD=0.70):
    """
    Runs V1 stress detection and intervention. Returns a formatted string.
    """
    output = f" 🤖 Model : BERT Sentiment Model(MVP)\n"
    output += f"--------------------------------------------------\n"

    stress_score, sentiment_label, confidence = detect_stress_score(forum_post)

    output += f"Model Analysis: Primary Sentiment='{sentiment_label}' with Confidence={confidence}\n"
    output += f"Calculated Stress Score: {round(stress_score, 4)} (Threshold: {STRESS_THRESHOLD})\n"

    if stress_score >= STRESS_THRESHOLD:
        output += f"🚨 STRESS DETECTED! Intervention Triggered.\n"
    else:
        output += "✅ Stress score is low. Monitoring continues.\n"

    output += provide_recommendation(stress_score, STRESS_THRESHOLD)
    return output


# --- Model V2 Functions (Advanced LLM with Fallback) ---
# Initialize V2 with a try/except block for stability

try:
    print("Initializing Advanced LLM Model (V2)... This may take a few minutes on first run.")
    stress_llm = pipeline(
        "text-generation",
        model="HuggingFaceH4/zephyr-7b-beta",
        torch_dtype=torch.bfloat16,
        device_map="auto"
    )

    def detect_stress_llm(text):
        system_prompt = (
            "You are an AI stress monitoring system in an academic forum. "
            "Analyze the following student post for signs of stress, anxiety, or being overwhelmed. "
            "Respond ONLY with one of the following words: HIGH_STRESS, MEDIUM_STRESS, or LOW_STRESS. DO NOT include any other text."
        )

        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": text}
        ]

        # Use the generator and extract the classification word
        response = stress_llm(messages, max_new_tokens=5, do_sample=False, truncation=True)
        classification_text = response[0]['generated_text'][-1]['content'].strip().upper()

        if "HIGH_STRESS" in classification_text:
            return 0.95, "HIGH"
        elif "MEDIUM_STRESS" in classification_text:
            return 0.75, "MEDIUM"
        else:
            return 0.40, "LOW"

    def monitor_and_intervene_v2(forum_post, STRESS_THRESHOLD=0.65):
        """
        Runs V2 stress detection and intervention. Returns a formatted string.
        """
        output = f" ✨ Model : Advanced LLM Model\n"
        output += f"--------------------------------------------------\n"

        stress_score, level_label = detect_stress_llm(forum_post)

        output += f"LLM Classification: {level_label}\n"
        output += f"Calculated Stress Score:{round(stress_score, 4)} (Threshold: {STRESS_THRESHOLD})\n"

        if stress_score >= STRESS_THRESHOLD:
            output += f"🚨 STRESS DETECTED! Intervention Triggered.\n"
        else:
            output += "✅ Stress score is low. Monitoring continues.\n"

        output += provide_recommendation(stress_score, STRESS_THRESHOLD)
        return output

except Exception as e:
    # --- FALLBACK DEFINITION (V2) ---
    print(f"LLM Model V2 failed to load due to error: {e}")
    def monitor_and_intervene_v2(forum_post, STRESS_THRESHOLD=0.65):
        """Fallback function when V2 fails."""
        return (
            "❌ Model V2: Advanced LLM (FAILED TO LOAD)\n"
            "--------------------------------------------------\n"
            "🚨 Error: The Advanced LLM could not be initialized (e.g., GPU memory limits).\n"
            "Please demonstrate using the robust **Basic Sentiment Model (V1)**.\n"
            "We would use Google Vertex AI for production LLM deployment."
        )


# ----------------------------------------------------------------------
# PART 2: GRADIO INTERFACE (THE FRONT END)
# ----------------------------------------------------------------------

with gr.Blocks(theme=gr.themes.Soft(), title="Academic Forum Stress Detector") as demo:
    gr.Markdown(
    """
    <div style="text-align: center; margin-bottom: 20px;">
        <h1 style="color: #4CAF50;">🧠 Academic Forum Stress Detector 🧑‍🎓</h1>
        <p style="font-size: 1.1em; color: #555;">
            Leveraging AI to detect student stress and suggest supportive resources.
        </p>
        <hr style="border-top: 2px solid #eee; margin: 20px auto; width: 80%;">
    </div>
    """
    )

    with gr.Row():
        with gr.Column(scale=2):
            gr.Markdown(
            """
            ## 📝 Enter Student Post
            <p style="font-size: 0.9em; color: #777;">Paste a student's discussion forum entry below for analysis.</p>
            """
            )
            post_input = gr.Textbox(
                lines=7,
                placeholder="E.g., 'I'm completely overwhelmed with all these deadlines. I don't know where to start and feel like giving up.'",
                label="Student Forum Post"
            )

        with gr.Column(scale=3):
            gr.Markdown(
            """
            ## 💡 AI Analysis & Recommendations
            <p style="font-size: 0.9em; color: #777;">Results from our AI models, indicating detected stress and recommended interventions.</p>
            """
            )
            intervention_output = gr.Markdown(
                label="Stress Level & Support Recommendation",
                value="<p style='color: #888; text-align: center; padding: 20px; border: 1px dashed #ccc; border-radius: 8px;'>Click a model button below to analyze the post!</p>"
            )

    gr.Markdown(
    """
    <h2 style="text-align: center; margin-top: 30px; color: #007BFF;">Choose Your AI Model</h2>
    <p style="text-align: center; font-size: 0.9em; color: #555;">Compare the performance of a robust baseline model versus a specialized Large Language Model.</p>
    """
    )
    with gr.Row(variant="panel"):
        with gr.Column():
            btn_v1 = gr.Button(
                "🚀 Run: Bert Sentiment Model (MVP)",
                variant="secondary"
            )
            gr.Markdown(
            """
            <p style="text-align: center; font-size: 0.8em; color: #999;"><i>BERT-based, fast, and robust for a reliable baseline.</i></p>
            """
            )
        with gr.Column():
            btn_v2 = gr.Button(
                "🌟 Run: Advanced LLM Model",
                variant="primary"
            )
            gr.Markdown(
            """
            <p style="text-align: center; font-size: 0.8em; color: #999;"><i>Mental-health-aware LLM for nuanced context analysis.</i></p>
            """
            )

    # --- Define Event Listeners (Tying buttons to the correct functions) ---
    btn_v1.click(
        fn=monitor_and_intervene_v1,
        inputs=[post_input],
        outputs=[intervention_output]
    )

    btn_v2.click(
        fn=monitor_and_intervene_v2,
        inputs=[post_input],
        outputs=[intervention_output]
    )

    gr.Markdown(
    """
    <div style="text-align: center; margin-top: 40px; padding-top: 20px; border-top: 1px solid #eee;">
        <p style="font-size: 0.8em; color: #aaa;">Developed for the Hackathon | Powered by Google Colab & Hugging Face Transformers | Target Deployment: Google Cloud Platform (GCP)</p>
    </div>
    """
    )

# Launch the interface
demo.launch(share=True)




Initializing Basic Sentiment Model (V1)...


emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0
Device set to use cuda:0


Initializing Advanced LLM Model (V2)... This may take a few minutes on first run.


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Device set to use cuda:0


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5c07212c75cc623485.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
